TODO:
 * Stepper motor skipping steps--more current (better power supply)?
 * Normalize action and observation spaces (see: https://ai.stackexchange.com/questions/21477/why-do-we-also-need-to-normalize-the-actions-values-on-continuous-action-spaces)

In [3]:
# !python -m pip install gymnasium==0.28.1
# !python -m pip install stable-baselines3[extra]==2.1.0
# !python -m pip install ax-platform==0.3.4
# !python -m pip install wandb

In [4]:
# Check versions
import importlib.metadata

print(f"torch version: {importlib.metadata.version('torch')}")
print(f"gymnasium version: {importlib.metadata.version('gymnasium')}")
print(f"sb3 version: {importlib.metadata.version('stable-baselines3')}")
print(f"cv2 version: {importlib.metadata.version('opencv-python')}")
print(f"ax version: {importlib.metadata.version('ax-platform')}")

torch version: 2.0.0
gymnasium version: 0.28.1
sb3 version: 2.1.0
cv2 version: 4.7.0.68
ax version: 0.3.4


In [5]:
# Python Standard Library
import time
import datetime
import os
import random
import logging
import math
import csv
from typing import Any, Dict, Tuple, Union

# Encoder and stepper controls (local)
from control_comms import ControlComms, StatusCode, DebugLevel

# Third-party packages
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import wandb

# Reinforcement model modules
import stable_baselines3 as sb3
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import KVWriter, Logger

# Meta Ax
from ax.service.ax_client import AxClient
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.utils.tutorials.cnn_utils import train, evaluate

## Settings

In [6]:
# Communication settings
SERIAL_PORT = "COM4"    # Check your devices
BAUD_RATE = 500_000   # Must match what's in the Arduino code!
CTRL_TIMEOUT = 2.0      # Seconds
DEBUG_LEVEL = DebugLevel.DEBUG_ERROR

# Reinforcement learning settings
K_T = 5                 # Reward constant to multiply theta (angle of encoder)
K_DT = 0.05             # Reward constant to multiply dtheto/dt (angular velocity of encoder)
K_P = 2                 # Reward constant to multiply phi (angle of stepper)
K_DP = 0.05             # Reward constant to multiply dphi/dt (angular velocity of stepper)
REWARD_OOB = -500       # Reward (penalty) for having the stepper motor move out of bounds (OOB)
ENC_ANGLE_NORM = 180    # Divide by this to normalize +/-180 deg angle to +/-1
STP_MOVE_MIN = -10.0
STP_MOVE_MAX = 10.0
STP_ANGLE_MIN = -180    # Episode ends if stepper goes beyond this angle
STP_ANGLE_MAX = 180     # Episode ends if stepper goes beyond this angle
STP_ANGLE_NORM = 180    # Divide by this to normalize +/-180 deg angle to +/-1

ENV_TIMEOUT = 30.0
RESET_SETTLE_TIME = 2.0 # Seconds to wait after reset to start moving again

# Angle constants
ENC_OFFSET = 180.0      # Pendulum in the "up" position should be 0 deg
ANG_REV = 360           # Degrees in a single revolution

In [7]:
# Communication constants
CMD_SET_HOME = 0        # Set current stepper position as home (0 deg)
CMD_MOVE_TO = 1         # Move stepper to a particular position (deg)
CMD_MOVE_BY = 2         # Move stepper by a given amount (deg)
CMD_SET_STEP_MODE = 3   # Set step mode
CMD_SET_BLOCK_MODE = 4  # Set blocking mode
CMD_NOP = 5             # Take no action, just receive observation
STEP_MODE_1 = 0         # 1 division per step
STEP_MODE_2 = 1         # 2 divisions per step
STEP_MODE_4 = 2         # 4 divisions per step
STEP_MODE_8 = 3         # 8 divisions per step
STEP_MODE_16 = 4        # 16 divisions per step
STATUS_OK = 0           # Stepper idle
STATUS_STP_MOVING = 1   # Stepper is currently moving

# Set to desired step mode
STEP_MODE = STEP_MODE_8

## Setup

In [56]:
# Close connection to Arduino board (if open)
try:
    controller.close()
except:
    pass

In [57]:
# Connect to Arduino board
controller = ControlComms(timeout=CTRL_TIMEOUT, debug_level=DEBUG_LEVEL)
ret = controller.connect(SERIAL_PORT, BAUD_RATE)
if ret is not StatusCode.OK:
    print("ERROR: Could not connect to board")

In [58]:
# Test basic comms
controller.step(CMD_SET_STEP_MODE, [STEP_MODE_8])
controller.step(CMD_SET_HOME, [0])
controller.step(CMD_SET_BLOCK_MODE, [0])
controller.step(CMD_MOVE_BY, [-25])

(1, 178555, False, [0.0, 0.0])

In [59]:
# Numpy test
action = np.array([[-25]])
action_list = action.flatten().tolist()
controller.step(CMD_MOVE_BY, action_list)

(1, 179710, False, [8.4, -24.75])

In [60]:
# Test hard limit (360 deg)
for i in range(20):
    resp = controller.step(CMD_MOVE_BY, [50])
    print(resp)
    time.sleep(0.1)

(1, 180577, False, [11.1, -49.5])
(1, 180695, False, [5.4, -44.1])
(1, 180822, False, [349.8, -30.6])
(1, 180947, False, [335.1, -9.0])
(1, 181071, False, [332.4, 13.27])
(1, 181196, False, [345.9, 36.45])
(1, 181320, False, [357.0, 58.5])
(1, 181445, False, [6.0, 81.0])
(1, 181571, False, [12.6, 104.18])
(1, 181696, False, [15.6, 126.22])
(1, 181821, False, [10.2, 149.4])
(1, 181944, False, [359.7, 172.12])
(1, 182069, False, [351.3, 194.4])
(1, 182194, False, [347.1, 216.9])
(1, 182318, False, [348.3, 240.07])
(1, 182443, False, [354.3, 263.25])
(1, 182567, False, [2.7, 286.2])
(1, 182692, False, [9.6, 308.48])
(1, 182818, False, [11.7, 332.1])
(1, 182943, False, [13.5, 348.75])


In [62]:
# Stress/torque test
resp = controller.step(CMD_MOVE_TO, [0])
controller.step(CMD_SET_BLOCK_MODE, [1])
print(resp)
time.sleep(2.0)
action = 180
for i in range(10):
    action = -180 if action == 180 else 180
    resp = controller.step(CMD_MOVE_BY, [action])
    print(resp)
    time.sleep(0.01)

(0, 267023, False, [27.9, 1.12])
(0, 270065, False, [50.4, 0.0])
(0, 271075, False, [279.6, -179.77])
(0, 272104, False, [262.8, 0.22])
(0, 273106, False, [231.6, -179.55])
(0, 274133, False, [287.1, 0.45])
(0, 275135, False, [346.8, -179.32])
(0, 276164, False, [223.8, 0.68])
(0, 277173, False, [284.7, -179.1])
(0, 278203, False, [107.1, 0.9])
(0, 279217, False, [62.1, -178.87])


In [26]:
# Comms stress test
# resp = controller.step(CMD_MOVE_TO, [0])
# print(resp)
# time.sleep(2.0)
# for i in range(100000):
#     resp = controller.step(CMD_MOVE_BY, [0])
#     time.sleep(0.001)

In [63]:
# Move home
resp = controller.step(CMD_MOVE_TO, [0])
print(resp)

(0, 284730, False, [46.2, 1.12])


In [28]:
# Close comms
controller.close()

In [64]:
# Basic step test
action = np.array([[-10]])

controller = ControlComms(timeout=CTRL_TIMEOUT, debug_level=DEBUG_LEVEL)
ret = controller.connect(SERIAL_PORT, BAUD_RATE)
if ret is not StatusCode.OK:
    print("ERROR: Could not connect to board")

resp = controller.step(CMD_SET_BLOCK_MODE, [1])
print(resp)
resp = controller.step(CMD_SET_BLOCK_MODE, [1])
print(resp)
    
# Reset
time.sleep(2.0)
resp = controller.step(CMD_MOVE_TO, [0.0])
print(resp)
time.sleep(2.0)

# Loop
for i in range(10):
    action_list = action.flatten().tolist()
    resp = controller.step(CMD_MOVE_BY, action_list)
    print(resp)
    time.sleep(0.1)

# Move home and close
resp = controller.step(CMD_MOVE_TO, [0])
print(resp)
controller.close()

(0, 308197, False, [0.0, 0.0])
(0, 308217, False, [0.0, 0.0])
(0, 310232, False, [0.0, 0.0])
(0, 312465, False, [0.0, 0.0])
(0, 312792, False, [359.4, -9.68])
(0, 313119, False, [355.5, -19.35])
(0, 313431, False, [1.8, -29.02])
(0, 313759, False, [356.4, -38.7])
(0, 314087, False, [358.5, -48.38])
(0, 314414, False, [0.9, -58.05])
(0, 314740, False, [356.4, -67.72])
(0, 315066, False, [0.6, -77.4])
(0, 315396, False, [358.8, -87.07])
(0, 316226, False, [358.5, -96.75])


In [65]:
# Log in to Weights & Biases
wandb.login()

True

In [66]:
# Make wandb be quiet
os.environ["WANDB_SILENT"] = "true"
logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)

## Helper functions

In [32]:
def set_random_seeds(seed: int) -> None:
    """
    Seed the different random generators.
    https://stable-baselines3.readthedocs.io/en/master/_modules/stable_baselines3/common/utils.html
    """
    
    # Set seed for Python random and NumPy
    random.seed(seed)
    np.random.seed(seed)

In [33]:
def calc_angular_velocity(ang, ang_prev, dt):
    """
    Estimate engular velocity based on current and previous readings. Note that we assume that the
    object in question cannot go the long way around (e.g. more than 180 deg).
    """
    da = ang - ang_prev
    if da > (ANG_REV / 2):
        da -= ANG_REV
    elif da < -(ANG_REV / 2):
        da += ANG_REV
    
    return da / dt

## Build gym Environment

Subclass gymnasium.Env to create a custom environment. Learn more here:<br>
https://gymnasium.farama.org/tutorials/gymnasium_basics/environment_creation/

In [67]:
class Pendulum(gym.Env):
    """
    Subclass gymnasium Env class
    
    This is the gym wrapper class that allows our agent to interact with our environment. We need
    to implement four main methods: step(), reset(), render(), and close(). We should also define
    the action_space and observation space as class members.
    
    Note: on Windows, time.sleep() is only accurate to around 10ms. As a result, setting fps_limit
    will give you a "best effort" limit.
    
    More information: https://gymnasium.farama.org/api/env/
    """
    
    def __init__(
        self,
        serial_port,
        baud_rate,
        ctrl_timeout=1.0,
        debug_level=DebugLevel.DEBUG_NONE,
        env_timeout=0.0, 
        stp_mode=STEP_MODE_8, 
        stp_blocking=False
    ):
        """
        Set up the environment, action, and observation shapes. Optional tiemout in seconds.
        """
        
        # Call superclass's constructor
        super().__init__()
        
        # Connect to Arduino board
        self.ctrl = ControlComms(timeout=ctrl_timeout, debug_level=debug_level)
        try:
            self.ctrl.close()
        except:
            pass
        ret = self.ctrl.connect(serial_port, baud_rate)
        if ret is not StatusCode.OK:
            print("ERROR: Could not connect to board")
        
        # Define action space (scalar signifying how many degrees to move stepper by)
        self.action_space = gym.spaces.Box(
            low=STP_MOVE_MIN,
            high=STP_MOVE_MAX,
            shape=(1, 1),
            dtype=np.float32
        )
        
        # Define observation space 
        # [encoder angle, encoder angular velocity, stepper angle, stepper angular velocity]
        self.observation_space = gym.spaces.Box(
            low=np.array([-180, -np.inf, STP_ANGLE_MIN, -np.inf]),
            high=np.array([180, np.inf, STP_ANGLE_MAX, np.inf]),
            dtype=np.float32
        )
        
        # Record time from microcontroller and own elapsed time
        self.timestamp = 0
        self.timeout = env_timeout
        self.start_time = time.time()
        
        # Record previous encoder and stepper angles (to calculate velocities)
        self.angle_stp_prev = 0
        self.angle_enc_prev = 0
        
        # Set current stepper position as "home" and optionally set blocking
        self.ctrl.step(CMD_SET_STEP_MODE, [stp_mode])
        self.ctrl.step(CMD_SET_HOME, [0])
        if stp_blocking:
            self.ctrl.step(CMD_SET_BLOCK_MODE, [1])
        else:
            self.ctrl.step(CMD_SET_BLOCK_MODE, [0])
    
    def __del__(self):
        """
        Destructor: make sure to close the serial port
        """
        self.close()
    
    def step(self, action: np.ndarray):
        """
        What happens when you tell the stepper motor to do something then record the observation.
        """
        
        # Initialize return values
        obs = np.array([0.0, 0.0, 0.0, 0.0], dtype=np.float32)
        reward = 0.0
        info = {"error": False, "dtime": 0.0, "elapsed_time": 0.0}
        terminated = False
        truncated = False
        
        # Box is 2D NumPy array, action must be sent out as 1D list [...]
        action_list = action.flatten().tolist()
        
        # Move the stepper motor and wait for a response
        resp = self.ctrl.step(CMD_MOVE_BY, action_list)
        if resp:
            
            # Extract information from controller response
            status, timestamp, terminated, angles = resp
            
            # Compute lapsed time (in seconds) from previous observation (milliseconds)
            info["dtime"] = (timestamp - self.timestamp) / 1000.0
            self.timestamp = timestamp
            
            # Offset encoder angle so that 0 deg is up
            angles[0] -= ENC_OFFSET
            
            # Calculate velocities
            dtheta = calc_angular_velocity(angles[0], self.angle_enc_prev, info['dtime'])
            dphi = calc_angular_velocity(angles[1], self.angle_stp_prev, info['dtime'])
            self.angle_enc_prev = angles[0]
            self.angle_stp_prev = angles[1]
            
            # Construct observation (normalized)
            obs[0] = angles[0] / ENC_ANGLE_NORM
            obs[1] = dtheta / ENC_ANGLE_NORM
            obs[2] = angles[1] / STP_ANGLE_NORM
            obs[3] = dphi / STP_ANGLE_NORM
                    
            # Calculate reward if stepper is not out of bounds
            if (angles[1] >= STP_ANGLE_MIN) and (angles[1] <= STP_ANGLE_MAX):
                reward = -1 * (K_T * obs[0] ** 2 + 
                               K_DT * obs[1] ** 2 + 
                               K_P * obs[2] ** 2 +
                               K_DP * obs[3] ** 2)
            
            # Stepper motor is out of bounds--terminate episode
            else:
                reward = REWARD_OOB
                terminated = True
        
        # Something is wrong with communication
        else:
            print("ERROR: Could not communicate with Arduino")
            info["error"] = True
            terminated = True
        
        # Calculate elapsed time
        info["elapsed_time"] = time.time() - self.start_time
        
        # Check if we've exceeded the time limit
        if not terminated and self.timeout > 0.0 and info["elapsed_time"] >= self.timeout:
            truncated = True
        
        return obs, reward, terminated, truncated, info
    
    def reset(self, seed=None):
        """
        Return the pendulum to the starting position
        """
        
        # Initialize return values
        obs = np.array([0.0, 0.0, 0.0, 0.0], dtype=np.float32)
        info = {"error": False, "dtime": 0, "elapsed_time": 0.0}
        
        # Reset timer
        self.start_time = time.time()
        
        # Let the pendulum fall and return to the starting position
        time.sleep(RESET_SETTLE_TIME)
        resp = self.ctrl.step(CMD_MOVE_TO, [0.0])
        if resp:
            
            # Extract information from controller response
            status, timestamp, terminated, angles = resp
            
            # Compute lapsed time (in seconds) from previous observation (milliseconds)
            info["dtime"] = (timestamp - self.timestamp) / 1000.0
            self.timestamp = timestamp
            
            # Offset encoder angle so that 0 deg is up
            angles[0] -= ENC_OFFSET
            
            # Calculate velocities
            dtheta = calc_angular_velocity(angles[0], self.angle_enc_prev, info['dtime'])
            dphi = calc_angular_velocity(angles[1], self.angle_stp_prev, info['dtime'])
            self.angle_enc_prev = angles[0]
            self.angle_stp_prev = angles[1]
            
            # Construct observation (normalized)
            obs[0] = angles[0] / ENC_ANGLE_NORM
            obs[1] = dtheta / ENC_ANGLE_NORM
            obs[2] = angles[1] / STP_ANGLE_NORM
            obs[3] = dphi / STP_ANGLE_NORM
            
            # Let pendulum settle for a bit
            time.sleep(RESET_SETTLE_TIME)
            
        # Something is wrong with communication
        else:
            print("ERROR: Could not communicate with Arduino")
            info["error"] = True
            
        # Calculate elapsed time
        info["elapsed_time"] = time.time() - self.start_time
        
        return obs, info
    
    def close(self):
        """
        Close connection to Arduino
        """
        self.ctrl.close()

## Test gym Environment

Test the gym wrapper before training

In [75]:
# Create our environment
try:
    env.close()
except:
    pass
env = Pendulum(
        SERIAL_PORT,
        BAUD_RATE,
        ctrl_timeout=CTRL_TIMEOUT,
        debug_level=DEBUG_LEVEL,
        env_timeout=ENV_TIMEOUT, 
        stp_mode=STEP_MODE, 
        stp_blocking=False
)

In [76]:
# Test encoder
obs, info = env.reset()
obs_str = ", ".join([f"{val:.2f}" for val in obs])
if info["error"]:
    print("Stopping")
else:
    print(f"{'Step': ^8} | {'Observation': ^36} | {'Reward': ^8} | {'Done': ^8} | Info")
    print(f"{'Reset': ^8} | {obs_str: <36} | {0.0: <8} | {str(False): ^8} | {info}")
    for i in range(10):
        obs, reward, terminated, truncated, info = env.step(np.array([[0]]))
        obs_str = ", ".join([f"{val:.2f}" for val in obs])
        print(f"{i: ^8} | {obs_str: <36} | {reward: <8.2f} | {str(terminated or truncated): ^8} | {info}")
        if info["error"]:
            print("Stopping")
            break
        if terminated or truncated:
            print("Episode done")
            break
        time.sleep(1.0)

  Step   |             Observation              |  Reward  |   Done   | Info
 Reset   | -1.00, -0.00, 0.00, 0.00             | 0.0      |  False   | {'error': False, 'dtime': 504.447, 'elapsed_time': 4.026068687438965}
   0     | -1.00, 0.00, 0.00, 0.00              | -5.00    |  False   | {'error': False, 'dtime': 2.024, 'elapsed_time': 4.044030666351318}
   1     | -1.00, 0.00, 0.00, 0.00              | -5.00    |  False   | {'error': False, 'dtime': 1.026, 'elapsed_time': 5.067969083786011}
   2     | -1.00, 0.00, 0.00, 0.00              | -5.00    |  False   | {'error': False, 'dtime': 1.017, 'elapsed_time': 6.086517572402954}
   3     | -1.00, 0.00, 0.00, 0.00              | -5.00    |  False   | {'error': False, 'dtime': 1.027, 'elapsed_time': 7.11404824256897}
   4     | -1.00, 0.00, 0.00, 0.00              | -5.00    |  False   | {'error': False, 'dtime': 1.026, 'elapsed_time': 8.132933855056763}
   5     | -1.00, 0.00, 0.00, 0.00              | -5.00    |  False   | {'error': 

In [70]:
# Try running the environment for a few steps (stepper should move some)
obs, info = env.reset()
obs_str = ", ".join([f"{val:.2f}" for val in obs])
if info["error"]:
    print("Stopping")
else:
    print(f"{'Step': ^8} | {'Observation': ^36} | {'Reward': ^8} | {'Done': ^8} | Info")
    print(f"{'Reset': ^8} | {obs_str: <36} | {0.0: <8} | {str(False): ^8} | {info}")
    for i in range(10):
        obs, reward, terminated, truncated, info = env.step(np.array([[-25]]))
        obs_str = ", ".join([f"{val:.2f}" for val in obs])
        print(f"{i: ^8} | {obs_str: <36} | {reward: <8.2f} | {str(terminated or truncated): ^8} | {info}")
        if info["error"]:
            print("Stopping")
            break
        if terminated or truncated:
            print("Episode done")
            break
        time.sleep(0.1)

  Step   |             Observation              |  Reward  |   Done   | Info
 Reset   | -1.00, 0.00, 0.00, 0.00              | 0.0      |  False   | {'error': False, 'dtime': 22.324, 'elapsed_time': 4.031323432922363}
   0     | -1.00, 0.00, 0.00, 0.00              | -5.00    |  False   | {'error': False, 'dtime': 2.026, 'elapsed_time': 4.047344446182251}
   1     | -0.98, 0.16, -0.03, -0.25            | -4.81    |  False   | {'error': False, 'dtime': 0.124, 'elapsed_time': 4.167266368865967}
   2     | -0.94, 0.35, -0.11, -0.60            | -4.43    |  False   | {'error': False, 'dtime': 0.124, 'elapsed_time': 4.29517126083374}
   3     | -0.93, 0.05, -0.19, -0.65            | -4.42    |  False   | {'error': False, 'dtime': 0.126, 'elapsed_time': 4.421221494674683}
   4     | -0.96, -0.27, -0.27, -0.64           | -4.81    |  False   | {'error': False, 'dtime': 0.124, 'elapsed_time': 4.545549154281616}
   5     | 0.99, -0.42, -0.35, -0.65            | -5.12    |  False   | {'error': F

In [71]:
# Test timeout (stepper will reset and vibrate for a while)
obs, info = env.reset()
print(f"obs: {obs}, info: {info}")
action = 2
if not info["error"]:
    for i in range(1000):
        action = -2 if action == 2 else 2
        obs, reward, terminated, truncated, info = env.step(np.array([[action]]))
        # print(f"{i: ^8} | {obs_str: <36} | {reward: <8.2f} | {str(terminated or truncated): ^8} | {info}")
        if terminated or truncated:
            print("Episode done")
            break

obs: [-0.99333334 -0.0026336  -0.81       -0.00835663], info: {'error': False, 'dtime': 16.454, 'elapsed_time': 4.036940574645996}


In [72]:
# Final environment check to make sure it works with Stable-Baselines3 (no errors means it worked)
env_checker.check_env(env)

In [77]:
# Close the environment
env.close()

## Define Test

In [78]:
# Function that tests the model in the given environment
def test_agent(env, model, max_steps=0):

    # Reset environment
    obs, info = env.reset()
    ep_len = 0
    ep_rew = 0
    avg_step_time = 0.0

    # Run episode until complete
    while True:

        # Provide observation to policy to predict the next action
        timestamp = time.time()
        action, _ = model.predict(obs)

        # Perform action, update total reward
        obs, reward, terminated, truncated, info = env.step(action)
        avg_step_time += time.time() - timestamp
        ep_rew += reward

        # Increase step counter
        ep_len += 1
        if (max_steps > 0) and (ep_len >= max_steps):
            break

        # Check to see if episode has ended
        if terminated or truncated:
            break
        
    # Calculate average step time
    avg_step_time /= ep_len
    
    return ep_len, ep_rew, avg_step_time

## Testing and logging callbacks

Construct custom callbacks for Stable-Baselines3 to test our agent and log metrics to Weights & Biases.

In [79]:
# Evaluate agent on a number of tests
def evaluate_agent(env, model, steps_per_test, num_tests):
    
    # Initialize metrics
    avg_ep_len = 0
    avg_ep_rew = 0
    avg_step_time = 0.0
    
    # Test the agent a number of times
    for ep in range(num_tests):
        ep_len, ep_rew, step_time = test_agent(env, model, max_steps=steps_per_test)
        avg_ep_len += ep_len
        avg_ep_rew += ep_rew
        avg_step_time += step_time
        
    # Compute metrics
    avg_ep_len /= num_tests
    avg_ep_rew /= num_tests
    avg_step_time /= num_tests
    
    return avg_ep_len, avg_ep_rew, avg_step_time

In [80]:
class EvalAndSaveCallback(BaseCallback):
    """
    Evaluate and save the model every ``check_freq`` steps
    
    More info: https://stable-baselines3.readthedocs.io/en/master/guide/callbacks.html
    """
    
    # Constructor
    def __init__(
        self, 
        check_freq, 
        save_dir,
        model_name="model",
        replay_buffer_name=None,
        steps_per_test=0, 
        num_tests=10,
        step_offset=0,
        verbose=1,
    ):
        super(EvalAndSaveCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_dir = save_dir
        self.model_name = model_name
        self.replay_buffer_name = replay_buffer_name
        self.num_tests = num_tests
        self.steps_per_test = steps_per_test
        self.step_offset = step_offset
        self.verbose = verbose
        
    # Create directory for saving the models
    def _init_callback(self):
        if self.save_dir is not None:
            os.makedirs(self.save_dir, exist_ok=True)
            
    # Save and evaluate model at a set interval
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            
            # Set actual number of steps (including offset)
            actual_steps = self.step_offset + self.n_calls
            
            # Save model
            model_path = os.path.join(self.save_dir, f"{self.model_name}_{str(actual_steps)}")
            self.model.save(model_path)
            
            # Save replay buffer
            if self.replay_buffer_name != None:
                replay_buffer_path = os.path.join(self.save_dir, f"{self.replay_buffer_name}")
                self.model.save_replay_buffer(replay_buffer_path)
            
            # Evaluate the agent
            avg_ep_len, avg_ep_rew, avg_step_time = evaluate_agent(
                env, 
                self.model, 
                self.steps_per_test, 
                self.num_tests
            )
            if self.verbose:
                print(f"{str(actual_steps)} steps | average test length: {avg_ep_len}, average test reward: {avg_ep_rew}")
                
            # Log metrics to WandB
            log_dict = {
                'avg_ep_len': avg_ep_len,
                'avg_ep_rew': avg_ep_rew,
                'avg_step_time': avg_step_time,
            }
            wandb.log(log_dict, commit=True, step=actual_steps)
            
        return True

In [81]:
class WandBWriter(KVWriter):
    """
    Log metrics to Weights & Biases when called by .learn()
    
    More info: https://stable-baselines3.readthedocs.io/en/master/_modules/stable_baselines3/common/logger.html#KVWriter
    """
    
    # Initialize run
    def __init__(self, run, verbose=1):
        super().__init__()
        self.run = run
        self.verbose = verbose

    # Write metrics to W&B project
    def write(self, 
              key_values: Dict[str, Any], 
              key_excluded: Dict[str, Union[str, Tuple[str, ...]]], 
              step: int = 0) -> None:
        log_dict = {}
        
        # Go through each key/value pairs
        for (key, value), (_, excluded) in zip(
            sorted(key_values.items()), sorted(key_excluded.items())):
            
            if self.verbose >= 2:
                print(f"step={step} | {key} : {value} ({type(value)})")
            
            # Skip excluded items
            if excluded is not None and "wandb" in excluded:
                continue
                
            # Log integers and floats
            if isinstance(value, np.ScalarType):
                if not isinstance(value, str):
                    wandb.log(data={key: value}, step=step)
                    log_dict[key] = value
                
        # Print to console
        if self.verbose >= 1:
            print(f"Log for steps={step}")
            print(f"--------------")
            for (key, value) in sorted(log_dict.items()):
                print(f"  {key}: {value}")
            print()
                
    # Close the W&B run
    def close(self) -> None:
        self.run.finish()

## Define train and test function for a single trial

A single "trial" is fully training and then testing the agent using one set of hyperparameters.

In [82]:
def do_trial(settings, hparams):
    """
    Training loop used to evaluate a set of hyperparameters
    """
    
    # Set random seed
    set_random_seeds(settings['seed'])
    
    # Create new W&B run
    config = {}
    dt = datetime.datetime.now(datetime.timezone.utc)
    dt = dt.replace(microsecond=0, tzinfo=None)
    run = wandb.init(
        project=settings['wandb_project'], 
        name=str(dt), 
        config=config,
        settings=wandb.Settings(silent=(not settings['verbose_wandb']))
    )

    # Print run info
    if settings['verbose_trial'] > 0:
        print(f"WandB run ID: {run.id}")
        print(f"WandB run name: {run.name}")
    
    # Log hyperparameters to W&B
    wandb.config.update(hparams)
    
    # Set custom logger with our custom writer
    wandb_writer = WandBWriter(run, verbose=settings['verbose_log'])
    loggers = Logger(
        folder=None,
        output_formats=[wandb_writer]
    )
    
    # Calculate derived hyperparameters
    n_steps = 2 ** hparams['steps_per_update_pow2']
    minibatch_size = 2 ** hparams['minibatch_size_pow2']
    layer_1 = 2 ** hparams['layer_1_pow2']
    layer_2 = 2 ** hparams['layer_2_pow2']

    # Create new agent
    # PPO docs: https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html
    # Policy networks: https://stable-baselines.readthedocs.io/en/master/modules/policies.html
    model = sb3.PPO(
        'MlpPolicy',
        env,
        learning_rate=hparams['learning_rate'], # Learning rate of neural network (default: 0.0003)
        n_steps=n_steps,                        # Number of steps per update (default: 2048)
        batch_size=minibatch_size,              # Minibatch size for NN update (default: 64)
        gamma=hparams['gamma'],                 # Discount factor (default: 0.99)
        gae_lambda=hparams['gae_lambda'],       # Trade-off of bias vs. variance for GAE (default: 0.95)
        clip_range=hparams['clip_range'],       # Clipping parameter (default: 0.2)
        ent_coef=hparams['entropy_coef'],       # Entropy, how much to explore (default: 0.0)
        vf_coef=hparams['vf_coef'],             # Value function coefficient for the loss calculation (default: 0.5)
        max_grad_norm=hparams['max_grad_norm'], # Max value for gradient clipping (default: 0.5)
        use_sde=hparams['use_sde'],             # Use generalized State Dependent Exploration (default: False)
        sde_sample_freq=hparams['sde_freq'],    # Number of steps before sampling new noise matrix (default -1)
        policy_kwargs={'net_arch': [layer_1, layer_2]}, # (default: [64, 64])
        verbose=settings['verbose_train']       # Print training metrics (default: 0)
    )
    steps_to_complete = settings['total_steps']
        
    # Set up checkpoint callback
    checkpoint_callback = EvalAndSaveCallback(
        check_freq=settings['checkpoint_freq'], 
        save_dir=settings['save_dir'],
        model_name=settings['model_name'],
        replay_buffer_name=settings['replay_buffer_name'],
        steps_per_test=settings['steps_per_test'],
        num_tests=settings['tests_per_check'],
        step_offset=(settings['total_steps'] - steps_to_complete),
        verbose=settings['verbose_test'],
    )
    
    # Choo choo train
    model.learn(total_timesteps=steps_to_complete, 
                callback=[checkpoint_callback])
    
    # Get dataframe of run metrics
    history = wandb.Api().run(f"{run.project}/{run.id}").history()

    # Get index of evaluation with maximum reward
    max_idx = np.argmax(history.loc[:, 'avg_ep_rew'].values)

    # Find number of steps required to produce that maximum reward
    max_rew_steps = history['_step'][max_idx]
    if settings['verbose_trial'] > 0:
        print(f"Steps with max reward: {max_rew_steps}")
    
    # Load model with maximum reward from previous run
    model_path = os.path.join(settings['save_dir'], f"{settings['model_name']}_{str(max_rew_steps)}.zip")
    model = sb3.PPO.load(model_path, env)
    
    # Evaluate the agent
    avg_ep_len, avg_ep_rew, avg_step_time = evaluate_agent(
        env, 
        model, 
        settings['steps_per_test'],
        settings['tests_per_check'],
    )
    
    # Log final evaluation metrics to WandB run
    wandb.run.summary['Average test episode length'] = avg_ep_len
    wandb.run.summary['Average test episode reward'] = avg_ep_rew
    wandb.run.summary['Average test step time'] = avg_step_time
    
    # Print final run metrics
    if settings['verbose_trial'] > 0:
        print("---")
        print(f"Best model: {settings['model_name']}_{str(max_rew_steps)}.zip")
        print(f"Average episode length: {avg_ep_len}")
        print(f"Average episode reward: {avg_ep_rew}")
        print(f"Average step time: {avg_step_time}")
                      
    # Close W&B run
    run.finish()
    
    return avg_ep_rew

## Perform trials

In [83]:
# Project settings that do not change
settings = {
    'wandb_project': "pendulum-esp32-hpo-1",
    'model_name': "ppo-pendulum",
    'ax_experiment_name': "ppo-pendulum-esp32-1",
    'ax_objective_name': "avg_ep_rew",
    'replay_buffer_name': None,
    'save_dir': "checkpoints",
    'checkpoint_freq': 10_000,
    'steps_per_test': 500,
    'tests_per_check': 10,
    'total_steps': 100_000,
    'num_trials': 100,
    'seed': 42,
    'verbose_ax': False,
    'verbose_wandb': False,
    'verbose_train': 0,
    'verbose_log': 0,
    'verbose_test': 0,
    'verbose_trial': 1,
}

In [84]:
# Define hyperparameters we want to optimize
# Ref: https://github.com/facebook/Ax/blob/6443cee30cbf8cec290200a7420a3db08e4b5445/ax/service/ax_client.py#L236
# Example: https://github.com/facebook/Ax/blob/main/tutorials/tune_cnn_service.ipynb
# Hyperparameters: https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html#stable_baselines3.ppo.PPO
hparams = [
    {
        'name': "n_envs",
        'type': "fixed",
        'value_type': "int",
        'value': 1,
    },
    {
        'name': "learning_rate",
        'type': "range",
        'value_type': "float",
        'bounds': [0.0005, 0.007],
        'log_scale': False,
    },
    {
        'name': "steps_per_update_pow2",
        'type': "range",
        'value_type': "int",
        'bounds': [6, 9], # Inclusive, 2**n between [64, 512]
        'log_scale': False,
        'is_ordered': False,
    },
    {
        'name': "minibatch_size_pow2",
        'type': "range",
        'value_type': "int",
        'bounds': [9, 12], # Inclusive, 2**n between [512, 4096]
        'log_scale': False,
        'is_ordered': False,
    },
    {
        'name': "gae_lambda",
        'type': "range",
        'value_type': "float",
        'bounds': [0.9, 0.99],
        'log_scale': False,
    },
    {
        'name': "clip_range",
        'type': "range",
        'value_type': "float",
        'bounds': [0.1, 0.4],
        'log_scale': False,
    },
    {
        'name': "gamma",
        'type': "range",
        'value_type': "float",
        'bounds': [0.92, 0.99],
        'log_scale': False,
    },
    {
        'name': "entropy_coef",
        'value_type': "float",
        'type': "range",
        'bounds': [0.0, 0.01],
        'log_scale': False,
    },
    {
        'name': "vf_coef",
        'type': "range",
        'value_type': "float",
        'bounds': [0.2, 0.7],
        'log_scale': False,
    },
    {
        'name': "max_grad_norm",
        'type': "range",
        'value_type': "float",
        'bounds': [0.5, 5.0],
        'log_scale': False,
    },
    {
        'name': "use_sde",
        'type': "fixed",
        'value_type': "bool",
        'value': True,
    },
    {
        'name': "sde_freq",
        'type': "range",
        'value_type': "int",
        'bounds': [2, 6],
        'log_scale': False,
    },
    {
        'name': "layer_1_pow2",
        'type': "fixed",
        'value_type': "int",
        'value': 8, # 2**n (is 256)
        'log_scale': False,
        'is_ordered': False,
    },
    {
        'name': "layer_2_pow2",
        'type': "fixed",
        'value_type': "int",
        'value': 8, # 2**n (is 256)
        'log_scale': False,
        'is_ordered': False,
    },
]

# Set parameter constraints
# Example: https://github.com/facebook/Ax/issues/621
parameter_constraints = [
    "minibatch_size_pow2 >= steps_per_update_pow2" # `batch_size` should be a factor of `n_steps * n_envs`, assume n_envs=1
]

In [85]:
# Create our environment
try:
    env.close()
except:
    pass
env = Pendulum(
        SERIAL_PORT,
        BAUD_RATE,
        ctrl_timeout=CTRL_TIMEOUT,
        debug_level=DEBUG_LEVEL,
        env_timeout=ENV_TIMEOUT, 
        stp_mode=STEP_MODE, 
        stp_blocking=False
)

In [86]:
# Cosntruct path to Ax experiment snapshot file
ax_snapshot_path = os.path.join(settings['save_dir'], f"{settings['ax_experiment_name']}.json")

In [87]:
# DANGER! Uncomment to delete the experiment file to start over

# os.remove(ax_snapshot_path)

In [88]:
# Load experiment from snapshot if it exists, otherwise create a new one
# Ref: https://ax.dev/versions/0.2.10/api/service.html#ax.service.ax_client.AxClient.create_experiment
if os.path.exists(ax_snapshot_path):
    print(f"Loading experiment from snapshot: {ax_snapshot_path}")
    ax_client = AxClient.load_from_json_file(ax_snapshot_path)
else:
    print(f"Creating new experiment. Snapshot to be saved at {ax_snapshot_path}.")
    ax_client = AxClient(
        random_seed=settings['seed'],
        verbose_logging=settings['verbose_ax'],
    )
    ax_client.create_experiment(
        name=settings['ax_experiment_name'],
        parameters=hparams,
        objective_name=settings['ax_objective_name'],
        minimize=False,
        parameter_constraints=parameter_constraints,
    )

[WARNING 10-02 15:02:22] ax.service.ax_client: Random seed set to 42. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 10-02 15:02:22] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[FixedParameter(name='n_envs', parameter_type=INT, value=1), RangeParameter(name='learning_rate', parameter_type=FLOAT, range=[0.0005, 0.007]), RangeParameter(name='steps_per_update_pow2', parameter_type=INT, range=[6, 9]), RangeParameter(name='minibatch_size_pow2', parameter_type=INT, range=[9, 12]), RangeParameter(name='gae_lambda', parameter_type=FLOAT, range=[0.9, 0.99]), RangeParameter(name='clip_range', parameter_type=FLOAT, range=[0.1, 0.4]), RangeParameter(name='gamma', parameter_type=FLOAT, range=[0.92, 

Creating new experiment. Snapshot to be saved at checkpoints\ppo-pendulum-esp32-1.json.


In [89]:
# DANGER! Use this cell to mark trials as failed (e.g. if component breaks and WandB shows bad data for a given trial)
# Check .json file with a site like https://jsonformatter.org/json-pretty-print

# trial_index = 8
# trial = ax_client.experiment.trials[trial_index]
# trial.mark_failed(unsafe=True)
# print(trial)
# ax_client.save_to_json_file(ax_snapshot_path)

In [ ]:
# Choo choo! Perform trials to optimize hyperparameters
while True:
    
    # Get next hyperparameters and end experiment if we've reached max trials
    next_hparams, trial_index = ax_client.get_next_trial()
    if trial_index >= settings['num_trials']:
        break
        
    # Show that we're starting a new trial
    if settings['verbose_trial'] > 0:
        print(f"--- Trial {trial_index} ---")
        
    # Perform trial
    avg_ep_rew = do_trial(settings, next_hparams)
    ax_client.complete_trial(
        trial_index=trial_index,
        raw_data=avg_ep_rew,
    )
    
    # Save experiment snapshot
    ax_client.save_to_json_file(ax_snapshot_path)

--- Trial 0 ---


WandB run ID: qzcbrowc
WandB run name: 2023-10-02 21:02:27


C:\Users\sgmustadio\anaconda3\envs\pytorch-gpu\lib\site-packages\stable_baselines3\ppo\ppo.py:148: UserWarning:

You have specified a mini-batch size of 4096, but because the `RolloutBuffer` is of size `n_steps * n_envs = 64`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 64
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=64 and n_envs=1)



## Analyze Top Performing Trials

In [ ]:
# Get runs in WandB project
runs = wandb.Api().runs(settings['wandb_project'])

In [ ]:
# Plot best average episode reward from each run over time
avg_rews = []
for i, run in enumerate(runs):
    avg_rew = run.summary['Average test episode reward']
    if isinstance(avg_rew, float):
        avg_rews.append(avg_rew)
avg_rews.reverse()
plt.plot(avg_rews)

In [ ]:
# CSV file path
csv_file_path = os.path.join(".", settings['wandb_project'] + ".csv")

# List summary names
summary_names = [
    "Average test episode reward",
    "Average test episode length",
    "Average test step time",
]

# Get hyperparameter names
hparam_names = [hparam['name'] for hparam in hparams]

print()

# Create CSV with HPO trial results
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["name"] + summary_names + hparam_names)
    for run in runs:
        row = [run.name]
        for name in summary_names:
            row.append(run.summary[name])
        for name in hparam_names:
            row.append(run.config[name])
        writer.writerow(row)